In [1]:
import pandas as pd
from google.cloud import bigquery
from google.cloud import storage
from datetime import date

In [2]:
client_bq = bigquery.Client()
client_storage = storage.Client()

In [8]:
q1="""

with pa as ( select policy_no, policy_holder_indiv_key, policy_rank, policy_plan_family_cd, conversion_dt, current_payment_to_dt, last_payment_method_id,policy_status_name, eti_rpu_ind, 
case when CURRENT_DATE() BETWEEN DATE_ADD(t1.current_payment_to_dt, INTERVAL 67+1 day) AND DATE_ADD(t1.current_payment_to_dt, INTERVAL 67+1+90 day) AND t1.eti_rpu_ind NOT IN ('RPU','ETI') then coalesce (last_scfl_pymnt_method_nm, last_payment_method_nm) else last_payment_method_nm end as new_payment_mentod_nm,
case when count(distinct policy_no) over(partition by policy_holder_indiv_key)=1 then 1
else first_policy_flag end new_first_policy_flag
from `merkle-gldp-client.analytics.policy_appl` t1
)
  select * from (
SELECT
  CASE
    WHEN UPPER(pa.policy_plan_family_cd) LIKE '%YOUNG%' THEN 'YA'
    WHEN UPPER(pa.policy_plan_family_cd) LIKE '%GROW UP%' OR UPPER(pa.policy_plan_family_cd) LIKE '%CHILDREN%' OR UPPER(pa.policy_plan_family_cd) LIKE '%STUDENT%' THEN 'GU'
    WHEN UPPER(pa.policy_plan_family_cd) LIKE '%ACCIDENT PROTECTION%' THEN 'AP' 
    WHEN UPPER(pa.policy_plan_family_cd) LIKE '%WHOLE%' OR UPPER(pa.policy_plan_family_cd) LIKE '%LIFE PAID%' OR UPPER(pa.policy_plan_family_cd) LIKE '%MONEYBACK%' OR UPPER(pa.policy_plan_family_cd) LIKE '%SENIOR LIFE%' OR UPPER(pa.policy_plan_family_cd) LIKE '%SL%' THEN 'WL'
    WHEN UPPER(pa.policy_plan_family_cd) LIKE '%COLLEGE%' THEN 'GLCP'
    WHEN UPPER(pa.policy_plan_family_cd) LIKE '%TERM%' OR UPPER(pa.policy_plan_family_cd) LIKE '%SPOUSE RIDER%' OR UPPER(pa.policy_plan_family_cd) LIKE '%LIFESTYLE%'THEN 'TL'
  ELSE 'Misc' END  AS product,
  CASE
    WHEN pa.conversion_dt BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 6 month)  AND CURRENT_DATE() THEN '0-6 months'
    WHEN pa.conversion_dt BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 12 month) AND DATE_SUB(CURRENT_DATE(), INTERVAL 6 month) THEN '6-12 months'
    WHEN pa.conversion_dt BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 3 year)   AND DATE_SUB(CURRENT_DATE(), INTERVAL 12 month) THEN '1-3 years'
    WHEN pa.conversion_dt BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 5 year)   AND DATE_SUB(CURRENT_DATE(), INTERVAL 3 year) THEN '3-5 years'
    WHEN pa.conversion_dt < DATE_SUB(CURRENT_DATE(), INTERVAL 5 year) THEN '5+ years'
    WHEN CURRENT_DATE() BETWEEN DATE_ADD(pa.current_payment_to_dt, INTERVAL 67+1 day) AND DATE_ADD(pa.current_payment_to_dt, INTERVAL 67+1+90 day) 
  AND pa.eti_rpu_ind NOT IN ('RPU','ETI') THEN 'Lapsed within 90 days'
  ELSE "other" END  AS time_group,
  CASE 
    WHEN new_payment_mentod_nm='Automated Clearing House' THEN 'Auto_billing' ELSE 'Not Auto-billing' END AS billing_method_indicator,
    case when  substring(inout_cd_id2_cd,2,1)='G' and current_Date() between current_date() and date_add(pa.current_payment_to_dt, interval 90 day) then 'Requested to cancel within 90 days' else 'Not canceled' end Requested_to_cancel,
     party.party_first_name, party.party_last_name,party.email_txt,party.cell_phone, pa.policy_no, pa.policy_status_name, pa.conversion_dt as start_dt, case WHEN CURRENT_DATE() BETWEEN DATE_ADD(pa.current_payment_to_dt, INTERVAL 67+1 day) AND DATE_ADD(pa.current_payment_to_dt, INTERVAL 67+1+90 day) 
 AND pa.eti_rpu_ind NOT IN ('RPU','ETI') THEN DATE_ADD(pa.current_payment_to_dt, INTERVAL 67+1 day) else null end as end_dt
FROM
  pa
  join `merkle-gldp-client.mdb.policy` pol on pol.policy_no=pa.policy_no and pol.policy_rank=pa.policy_rank
  join `merkle-gldp-client.mdb.party` party on party.party_id=pol.policy_holder_party_id 
  where 1=1
  and pa.new_first_policy_flag=1
  ) t1
  where 1=1
  and time_group!='other' and product!='Misc'

order by 1,2,3 desc
"""

In [9]:
%%time
df = client_bq.query(q1).to_dataframe()

CPU times: user 3min 11s, sys: 11.7 s, total: 3min 23s
Wall time: 11min 33s


In [10]:
print(df.shape)

(4757899, 12)


In [11]:
final_df=pd.DataFrame(columns=df.columns)
product=df['product'].unique()
time=df['time_group'].unique()
bill_m=df['billing_method_indicator'].unique()
canceled=df['Requested_to_cancel'].unique()

# bucket for testing
bucket = 'gs://merkle-gldp-sandbox/Maksim/'

# bucket for file delviery 
# It's still not clear what folder to use
# bucket = 'gs://merkle-gldp-client/outbound/landing/'
today=date.today().strftime("%Y%m%d")

In [12]:
sample_size=50 # initially we were asked to output 50 records per segment

for pr in product:
    for t in time:
        for bill in bill_m:
            for c in canceled:
                final_df=final_df.append(df[(df['product']==pr)&(df['time_group']==t)&(df['billing_method_indicator']==bill)\
                                        &(df['Requested_to_cancel']==c)].sample(frac=1).head(sample_size)) 

filename='{}_{}_data_{}.csv'.format('test','test_file_name',today) # to be update based on the provided filename
final_df.to_csv(bucket+filename, index=False)
print('file {} has been exported'.format(filename))
            # print(sample.shape)
            # print('selection {}, unique values in product {}'.format(pr,sample['product'].unique()))
            # print('selection {}, unique values in product {}'.format(t,sample['time_group'].unique()))
            # print('selection {}, unique values in product {}'.format(bill,sample['billing_method_indicator'].unique()))

file test_test_file_name_data_20230223.csv has been exported


In [16]:
final_df.shape

(3265, 11)